**Exercise No:1 - Apple mobility data**

In [ ]:
# 1st step import libraries
from pathlib import Path # To check files in colab folders
import pandas as pd  # dataFrames using pandas


In [ ]:
# 2en step fetch the url
# fetch the link of the COVID-19 and download "CSV" file
!wget https://covid19-static.cdn-apple.com/covid19-mobility-data/2210HotfixDev17/v3/en-us/applemobilitytrends-2022-03-27.csv
# CSV file will be downloaded and saved in the main folder in GOOGLE-COLAB repository



### Check if "applemobilitytrends" file is downloaded successfully
my_file = Path("applemobilitytrends-2022-03-27.csv")
if my_file.is_file():
  print("File Downloaded Successfully.")
else:
  print("Error Downloading File! Please check the URL or try again.")


In [ ]:
# 3rd step   
#Create a function that take no argument and return a dataframe with the following columns :
#-- geo_type, region, transportation_type ,alternative_name, sub-region, country, date, value

def getDataframe():
  df = pd.read_csv('applemobilitytrends-2022-03-27.csv')
  return df

print(getDataframe().head(10)) #-- Visualization: show 10 rows from dataFrame


**Exercise No:2 - FSA data**

In [ ]:

##################################### 
# In the following exercise, I used the "BeautifulSoup" library because it is very simple, very powerful, runs quickly, 
# and allows us to extract data from a badly written web page. It structures an HTML or XML web page, gets the data you
# need from amongst the lot, and allows you to extract the required data in the right format.    
#####################################


# 1st step install and import modules
import re
import requests
from bs4 import BeautifulSoup
import csv
from itertools import zip_longest
import pandas as pd

years_list = []

# 2nd step use requests to fetch the url
# fetch the link of the FSA Crop Acreage Data Reported to FSA and fetch in page details
url = "https://www.fsa.usda.gov/news-room/efoia/electronic-reading-room/frequently-requested-information/crop-acreage-data/index"
result = requests.get(url)

#3rd step save page content/markup 
src = result.content
#print(src)

# 4th step create soup object to parse content
soup = BeautifulSoup(src, "lxml")
#print(soup)

# 5th step find the elements containing info we need
#-- years, 
years = soup.find_all("div",{"class":"rxbodyfield"})

for i in range(len(years)):
  sp = BeautifulSoup(str(years[i]), "html.parser")
  year = sp.find_all("h3")

# a simple fucntion to get digits from a string
def only_numerics(seq):
    seq_type= type(seq)
    return seq_type().join(filter(seq_type.isdigit, seq))
    
#6th step loopover returned lists to extract needed info intolists
for i in range(len(year)-1):
           years_list.append(only_numerics(year[i].text))


# Final result List of years:
print("List Of Years:")
print(years_list)
print("  ")




### Extract "country_data" sheet, "country_data_links" and  "publication_date" function:
def extractDataFromWebsite(list_yr=[]): # default parameters: empty list  

  # Start Collecting DATA by year
  for i in range(len(years)):
    sp = BeautifulSoup(str(years[i]), "html.parser")
    data_conut = sp.find_all("li")


  data_link = []
  data_country =[]
  publication_date =[]
  ls_years =[]

  for i in range(len(data_conut)-1):
    try: 
      data_link.append('https://www.fsa.usda.gov' + data_conut[i].find("a").attrs['href'])
      data_country.append(data_conut[i].find("a").text) #.attrs['title'])

      strg = data_conut[i].find("a").text
      find_of = strg.find(' of ') + 4
      length_str = len(strg)
      if(strg.find(' of ') == -1):
        publication_date.append('None')
      else:
        publication_date.append(strg[find_of:length_str])


    except Exception:
      data_country.append(data_conut[i].text)
      data_link.append('None')
      publication_date.append('None')
      pass

 

  #create csv file andfill it with values
  file_list = [data_country, data_link, publication_date]
  exported = zip_longest(*file_list)

  with open("fsa_crop_acreage_data.csv", "w") as myfile:
      wr = csv.writer(myfile)
      wr.writerow(["Data Country","Data Link","Publication Date"])
      wr.writerows(exported)

  # CSV file created successfully
  print("CSV file created successfully...")
  print("Check default folder or wait for few seconds to see < fsa_crop_acreage_data.csv > File.")
  
  # Added Data dataframe
  df = pd.read_csv('fsa_crop_acreage_data.csv')

  # return dataframe as final result
  return df

# test extractDataFromWebsite() function
dataFrame = extractDataFromWebsite(years_list)
print("")
print(dataFrame)
